In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))
import pandas as pd
import numpy as np
import time
from preprocessing import *
from JREP import JREP

# set_seed(45)

In [2]:
df = pd.read_csv(r'..\output_csv\HCMUT-SuperNodeXP-2017-1.0.swf.csv')
# df = df.head(100)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15886 entries, 0 to 15885
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    15886 non-null  float64
 1   submit_time               15886 non-null  float64
 2   wait_time                 15886 non-null  float64
 3   run_time                  15886 non-null  float64
 4   num_allocated_processors  15886 non-null  float64
 5   avg_cpu_time_used         15886 non-null  float64
 6   used_memory               15886 non-null  float64
 7   requested_processors      15886 non-null  float64
 8   requested_time            15886 non-null  float64
 9   requested_memory          15886 non-null  float64
 10  status                    15886 non-null  float64
 11  user_id                   15886 non-null  float64
 12  group_id                  15886 non-null  float64
 13  executable_id             15886 non-null  float64
 14  queue_

In [3]:
df.head()

,job_id,submit_time,wait_time,run_time,num_allocated_processors,avg_cpu_time_used,used_memory,requested_processors,requested_time,requested_memory,status,user_id,group_id,executable_id,queue_id,partition_id,preceding_job_id,think_time
0,4.0,0.0,0.0,13.0,24.0,7.0,14761.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
1,8.0,40819.0,0.0,61.0,24.0,45.0,14761.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
2,10.0,45255.0,0.0,60.0,24.0,45.0,14760.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
3,11.0,45782.0,0.0,1.0,24.0,0.0,0.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
4,12.0,56510.0,0.0,1.0,24.0,0.0,0.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0


In [4]:
feature_columns = ['requested_processors', 'requested_time', 'avg_cpu_time_used', 'used_memory', 'submit_time', 'wait_time', 'user_id', 'group_id', 'executable_id', 'queue_id']
target_column = 'run_time'

X_train, X_test, Y_train, Y_test, scaler = prepare_data(df, feature_columns, target_column)

In [5]:
rmse_lst, mae_lst, mse_lst, r2_lst, infer_time_lst = [], [], [], [], []
for i in range(101):
    print(f"Iteration {i+1}/100")
    JREP_model = JREP()
    JREP_model.fit(X_train, Y_train)

    start = time.time()
    Y_pred = JREP_model.predict(X_test, scaler, len(feature_columns))
    end = time.time()
    rmse, mae, mse, r2 = JREP_model.evaluate_model(Y_test, Y_pred, scaler, len(feature_columns))

    rmse_lst.append(rmse)
    mae_lst.append(mae)
    mse_lst.append(mse)
    r2_lst.append(r2)
    infer_time_lst.append((end - start) / len(Y_pred))

    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"R2: {r2}")
    print(f'Inference Time: {(end - start)/len(Y_pred)} seconds')

# Save results to CSV
results_df = pd.DataFrame({
    'RMSE': rmse_lst,
    'MAE': mae_lst,
    'MSE': mse_lst,
    'R2': r2_lst,
    'Inference Time': infer_time_lst
})
results_df.to_csv('../output_HCMUT/JREP_results.csv', index=False)

Iteration 1/100
RMSE: 28972.855099805765
MAE: 12180.466133586657
MSE: 839426332.6343409
R2: 0.9361323925528879
Inference Time: 0.00028784122577933376 seconds
Iteration 2/100
RMSE: 27485.41665990739
MAE: 11438.580456592792
MSE: 755448128.9687148
R2: 0.9425218596655025
Inference Time: 0.00023209078367366346 seconds
Iteration 3/100
RMSE: 29685.48770686237
MAE: 11805.103631681071
MSE: 881228180.3942769
R2: 0.9329519061903538
Inference Time: 0.0002339832311452821 seconds
Iteration 4/100
RMSE: 31905.44541873477
MAE: 12789.58701232085
MSE: 1017957447.3678635
R2: 0.9225488835424994
Inference Time: 0.0002819303163262301 seconds
Iteration 5/100
RMSE: 28756.404285247874
MAE: 10948.398798897002
MSE: 826930787.4166224
R2: 0.9370831139512724
Inference Time: 0.00024498860503351967 seconds
Iteration 6/100
RMSE: 34554.83890445354
MAE: 12213.82487363968
MSE: 1194036891.7127361
R2: 0.9091519094499286
Inference Time: 0.000239193786022275 seconds
Iteration 7/100
RMSE: 32318.56145820138
MAE: 12118.772743601